### Load the train dataset.

In [22]:
import os
import random
import pandas as pd

news = []

for directory in os.listdir("1150haber/raw_texts/"):
    for filename in os.listdir("1150haber/raw_texts/" + directory):
        f = open("1150haber/raw_texts/" + directory + "/" + filename, "r", encoding="windows-1254")
        new = f.read().replace("\n", " ")
        new = new.lower()
        news.append(new)
        
random.shuffle(news)
print(len(news))

data = {"Haber": news}
df_train = pd.DataFrame(data)
df_train["Haber"] = df_train["Haber"].apply(str)
df_train.sample(20)

920


,Haber
210,"bi̇z daha ölmedi̇k"" galatasaray, levski sofy..."
188,nihat’ın zor sınavı i̇spanya ligi’nde li...
249,avea: şartlar gelişirse telsim'i mutlaka değer...
7,vagon fabrikası jean üretim merkezi oluyor t...
580,kırık tedavisinde yeni metot yüzüncü yıl üni...
850,milli güvenlik kurulu genel sekreterliği’nde d...
294,"kenan doğulu, 'metroseksüel' benzetmesinden ra..."
381,"erdoğan, kız yurdunda yemek yedi ..."
85,doğru yol’un ‘telefon sapığı’ dsp milletveki...
316,ralf ile derin'in nişantaşı molası ender mer...


### Load the test dataset.

In [23]:
import os
import random
import pandas as pd

news = []
labels = []
filenames = []
i = 0

for directory in os.listdir("1150haber/test/"):
    i += 1
    for filename in os.listdir("1150haber/test/" + directory):
        f = open("1150haber/test/" + directory + "/" + filename, "r", encoding="windows-1254")
        new = f.read().replace("\n", " ")
        new = new.lower()
        news.append(new)
        labels.append(i)
        
print(len(news))

data = {"Haber": news, "Sınıf": labels}
df_test = pd.DataFrame(data)
df_test["Haber"] = df_test["Haber"].apply(str)
df_test = df_test.sample(frac = 1).reset_index(drop=True)
df_test.sample(230).sort_index()

230


,Haber,Sınıf
0,"prof. dr. mehmet öz, sağlıklı yaşam için dikka...",3
1,nazan öncel coşkusu sahnelere altı yıl ara ...,2
2,ankara üniversitesi tıp fakültesi farmakoloji ...,3
3,"[habersağlık-i̇stanbul] mesleğinin baharında, ...",3
4,yıllardır televizyon kanallarına başarılı prog...,2
...,...,...
225,green card’ı çekecekti ‘green card’ını kaybett...,2
226,ssk’ya pahalı ilaç satarak devleti zarara uğra...,1
227,ankara - türk telekom'un indirim reklamları sa...,1
228,ytl'de sahteyi ayıklama makinasında yarış kızı...,1


### Background for labeling functions

In [24]:
para = ["lira", "dolar", "döviz", "faiz"]
ekonomi_kavramlar = ["cari", "enflasyon", "merkez bankası", "imf", "kredi", "yatırım"]
sayi_birimleri = ["milyon", "milyar", "bin", "trilyon", "katrilyon"]

magazin_kavramlar = ["ünlü", "tatil", "bodrum", "konser", "sevgili", "yakışıklı", "aşk"]
magazin_meslekler = ["manken", "oyuncu", "aktör"]
magazin_kisiler = ["hülya avşar", "beren saat", "ajda pekkan", "demet akalın", "mehmet ali erbil"]

vucut_bolumleri = ["kalp", "diş", "göz", "damar", "ağız", "cilt", "vücut", "yağ", "böbrek", "kan", "tırnak", "bağırsak", "baş"]
saglik_kavramlar = ["tedavi", "bilim", "ilaç", "teşhis", "hasta", "tıp", "ameliyat", "ssk", "hastane"]
hastaliklar = ["kanser", "şeker", "yağ", "hastalık", "doğum", "anne", "görme", "tansiyon"]

siyasi_rutbeler = ["genel başkanı", "milletvekili", "bakan", "başbakan", "cumhurbaşkanı", "dışişleri"]
siyasi_kavramlar = ["abd", "erdoğan", "kılıçdaroğlu", "demirtaş", "bahçeli", "akşener", "avrupa", "büyükşehir", "anayasa", "abd"]
siyasi_partiler = ["akp", "chp", "mhp", "hdp"]

spor_kavramlar = ["teknik", "milli", "gol", "takım", "maç", "transfer", "forma", "uefa", "direktör", "terim"]
takimlar = ["fenerbahçe", "galatasaray", "beşiktaş", "trabzonspor", "barcelona", "arjantin", "kocaelispor"]
dallar = ["futbolcu", "kupa", "saha", "hakem", "lig", "rakip", "şampiyon", "deplasman", "futbol", "basketbol", "penaltı"]

In [25]:
EKONOMI = 1
MAGAZIN = 2
SAGLIK = 3
SIYASI = 4
SPOR = 5
BELIRSIZ = -1

### Labeling functions.

In [26]:
from snorkel.labeling import labeling_function
import re

@labeling_function()
def para_iceriyor(sample_new):
    for para_tipi in para:
        if re.search(para_tipi, sample_new.Haber):
            return EKONOMI 
    return BELIRSIZ

@labeling_function()
def ekonomik_kavram_iceriyor(sample_new):
    for ekonomik_kavram in ekonomi_kavramlar:
         if re.search(ekonomik_kavram, sample_new.Haber):
            return EKONOMI
    return BELIRSIZ

@labeling_function()
def sayi_birimi_iceriyor(sample_new):
    for sayi_birimi in sayi_birimleri:
        if re.search(sayi_birimi, sample_new.Haber):
            return EKONOMI 
    return BELIRSIZ

@labeling_function()
def magazin_kavrami_iceriyor(sample_new):
    for magazin_kavrami in magazin_kavramlar:
        if re.search(magazin_kavrami, sample_new.Haber):
            return MAGAZIN 
    return BELIRSIZ

@labeling_function()
def magazin_meslegi_iceriyor(sample_new):
    for magazin_meslegi in magazin_meslekler:
        if re.search(magazin_meslegi, sample_new.Haber):
            return MAGAZIN 
    return BELIRSIZ

@labeling_function()
def magazin_kisiler_iceriyor(sample_new):
    for magazin_kisisi in magazin_kisiler:
        if re.search(magazin_kisisi, sample_new.Haber):
            return MAGAZIN 
    return BELIRSIZ

@labeling_function()
def vucut_bolumu_iceriyor(sample_new):
    for vucut_bolumu in vucut_bolumleri:
        if re.search(vucut_bolumu, sample_new.Haber):
            return SAGLIK 
    return BELIRSIZ

@labeling_function()
def saglik_kavramlari_iceriyor(sample_new):
    for saglik_kavrami in saglik_kavramlar:
        if re.search(saglik_kavrami, sample_new.Haber):
            return SAGLIK
    return BELIRSIZ

@labeling_function()
def hastalik_iceriyor(sample_new):
    for hastalik in hastaliklar:
        if re.search(hastalik, sample_new.Haber):
            return SAGLIK 
    return BELIRSIZ

@labeling_function()
def siyasi_rutbe_iceriyor(sample_new):
    for siyasi_rutbe in siyasi_rutbeler:
        if re.search(siyasi_rutbe, sample_new.Haber):
            return SIYASI 
    return BELIRSIZ

@labeling_function()
def siyasi_kavram_iceriyor(sample_new):
    for siyasi_kavram in siyasi_kavramlar:
        if re.search(siyasi_kavram, sample_new.Haber):
            return SIYASI 
    return BELIRSIZ

@labeling_function()
def siyasi_parti_iceriyor(sample_new):
    for siyasi_parti in siyasi_partiler:
        if re.search(siyasi_parti, sample_new.Haber):
            return SIYASI 
    return BELIRSIZ

@labeling_function()
def spor_kavramlar_iceriyor(sample_new):
    for spor_kavrami in spor_kavramlar:
        if re.search(spor_kavrami, sample_new.Haber):
            return SPOR 
    return BELIRSIZ

@labeling_function()
def takimlar_iceriyor(sample_new):
    for takim in takimlar:
        if re.search(takim, sample_new.Haber):
            return SPOR 
    return BELIRSIZ

@labeling_function()
def spor_dallari_iceriyor(sample_new):
    for spor_dali in dallar:
        if re.search(spor_dali, sample_new.Haber):
            return SPOR 
    return BELIRSIZ

In [27]:
from snorkel.labeling import PandasLFApplier

lfs = [para_iceriyor, ekonomik_kavram_iceriyor, sayi_birimi_iceriyor, magazin_kavrami_iceriyor, magazin_meslegi_iceriyor, 
       magazin_kisiler_iceriyor, vucut_bolumu_iceriyor, saglik_kavramlari_iceriyor, hastalik_iceriyor, siyasi_rutbe_iceriyor, 
       siyasi_kavram_iceriyor, siyasi_parti_iceriyor, spor_kavramlar_iceriyor, takimlar_iceriyor, spor_dallari_iceriyor]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

100%|███████████████████████████████████████████████████████████████████████████████| 920/920 [00:04<00:00, 213.80it/s]


In [29]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
para_iceriyor,0,[1],0.234783,0.234783,0.230435
ekonomik_kavram_iceriyor,1,[1],0.122826,0.121739,0.119565
sayi_birimi_iceriyor,2,[1],0.410870,0.409783,0.405435
magazin_kavrami_iceriyor,3,[2],0.547826,0.538043,0.528261
magazin_meslegi_iceriyor,4,[2],0.178261,0.175000,0.167391
magazin_kisiler_iceriyor,5,[2],0.021739,0.021739,0.017391
vucut_bolumu_iceriyor,6,[3],0.865217,0.846739,0.790217
saglik_kavramlari_iceriyor,7,[3],0.251087,0.247826,0.201087
hastalik_iceriyor,8,[3],0.271739,0.269565,0.227174
siyasi_rutbe_iceriyor,9,[4],0.265217,0.265217,0.264130


In [30]:
L_train.shape

(920, 15)

### Convert the labels from LFs into a single noise-aware probabilistic label per data point

In [31]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter(6)
preds_train = majority_model.predict(L=L_train)

In [32]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=6, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [33]:
Y_test = df_test.Sınıf.values

majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")
      
label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority Vote Accuracy:   76.1%
Label Model Accuracy:     75.2%


The **majority vote model** or more **sophisticated LabelModel** could in principle be used directly as a classifier if the outputs of our labeling functions were made available at test time. However, these models (i.e. these re-weighted combinations of our labeling function's votes) will *abstain on the data points that our labeling functions don't cover* (and additionally, may require slow or unavailable features to execute at test time).

In [34]:
from snorkel.labeling import filter_unlabeled_dataframe

probs_train = label_model.predict_proba(L=L_train)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=probs_train, L=L_train
)

In [35]:
probs_train.shape

(920, 6)

In [36]:
df_train.size

920

## Training a Classifier

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 5))
X_train = vectorizer.fit_transform(df_train_filtered.Haber.tolist())
X_test = vectorizer.transform(df_test.Haber.tolist())

In [51]:
from snorkel.utils import probs_to_preds

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

In [52]:
preds_train_filtered.shape

(909,)

### Logistic Regression

In [53]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver="liblinear")
sklearn_model.fit(X=X_train, y=preds_train_filtered)

print(f"Test Accuracy: {sklearn_model.score(X=X_test, y=Y_test) * 100:.1f}%")

Test Accuracy: 75.7%


### Naive Bayes

In [54]:
from sklearn.metrics import accuracy_score
from sklearn import naive_bayes

# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, preds_train_filtered)

# predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB, Y_test)*100)

Naive Bayes Accuracy Score ->  70.43478260869566


### SVM

In [55]:
from sklearn import svm
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, preds_train_filtered)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Y_test)*100)

SVM Accuracy Score ->  68.69565217391305
